In [1]:
!pip install pytorchvideo
!pip install accelerate -U
!pip install triton


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 42.7 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=7a9f75d2f6ff4484335f83f31a47c9acd0c36c1f0e15b119973e406c27be6604
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=042dcd213e964e60d62fafcdc35aa6e73a3b78bf70ce0ad02d25e1ee6c0d422f
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Cre

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_value_1 = user_secrets.get_secret("WANDB_API_KEY")

In [3]:
!git config --global credential.helper store

In [4]:
import huggingface_hub
huggingface_hub.login(secret_value_0,add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import wandb
wandb.login(key=secret_value_1)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
!git config --global credential.helper store

In [7]:
import huggingface_hub
huggingface_hub.login(secret_value_0,add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
!git config --global credential.helper store

In [9]:
import glob 
import pathlib
dataset_root_path =  "/kaggle/temp/UCF101_subset"
root_dir = "/kaggle/input/ucf-crime-full"
dest_dir = "/kaggle/temp/UCF101_subset"
root_dir = pathlib.Path(root_dir)

image_paths = glob.glob(f"{root_dir}/*/*.mp4")
class_labels = sorted({str(path).split("/")[4] for path in image_paths})
label2id = {label: i for i, label in enumerate(class_labels)}
id2label = {i: label for label, i in label2id.items()}
print(f"Unique classes: {list(label2id.keys())}.")

Unique classes: ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'Normal_Videos_for_Event_Recognition', 'RoadAccidents', 'Robbery', 'Shooting', 'Shoplifting', 'Stealing', 'Vandalism'].


In [10]:
import os
import shutil
import random
from pathlib import Path
from multiprocessing import Pool
import pathlib
import glob
import glob
import pathlib
import os
import shutil
import random
from concurrent.futures import ThreadPoolExecutor

# Set the paths
root_dir = "/kaggle/input/ucf-crime-full"
dest_dir = "/kaggle/temp/UCF101_subset"

root_dir = pathlib.Path(root_dir)
image_paths = glob.glob(f"{root_dir}/*/*.mp4")

# Create the destination directory
os.makedirs(dest_dir, exist_ok=True)

# Split the image paths into train, val, and test sets
random.seed(42)  # Set a random seed for reproducibility
random.shuffle(image_paths)
num_examples = len(image_paths)
train_split = int(0.7 * num_examples)
val_split = int(0.2 * num_examples)
test_split = num_examples - train_split - val_split

train_paths = image_paths[:train_split]
val_paths = image_paths[train_split:train_split+val_split]
test_paths = image_paths[train_split+val_split:]

# Create the train, val, and test directories
train_dir = Path(dest_dir, "train")
val_dir = Path(dest_dir, "val")
test_dir = Path(dest_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define a function to move a single file
def move_file(args):
    src_path, dest_dir, class_label = args
    class_dir = Path(dest_dir, id2label[label2id[str(class_label)]])
    os.makedirs(class_dir, exist_ok=True)
    shutil.copy(src_path, class_dir)

# Create a pool of worker processes
num_processes = os.cpu_count() or 6  # Use all available cores, or a minimum of 4
pool = Pool(num_processes)

# Move the files to the respective directories using the pool
tasks = []
for paths, split_dir in [(train_paths, train_dir), (val_paths, val_dir), (test_paths, test_dir)]:
    for path in paths:
        class_label = str(path).split("/")[4]
        tasks.append((path, split_dir, class_label))

pool.map(move_file, tasks)
pool.close()
pool.join()

print("Files have been moved to /kaggle/temp/UCF101_subset")

Files have been moved to /kaggle/temp/UCF101_subset


In [11]:
import random

# Shuffle the image paths randomly
random.shuffle(image_paths)

# Calculate the number of examples for each split
num_examples = len(image_paths)
train_split = int(0.7 * num_examples)  # 70% for training
val_split = int(0.2 * num_examples)  # 20% for validation
test_split = num_examples - train_split - val_split  # Remaining for testing

# Split the image paths into train, val, and test sets
train_paths = image_paths[:train_split]
val_paths = image_paths[train_split:train_split+val_split]
test_paths = image_paths[train_split+val_split:]

print(f"Number of training examples: {len(train_paths)}")
print(f"Number of validation examples: {len(val_paths)}")
print(f"Number of testing examples: {len(test_paths)}")

Number of training examples: 700
Number of validation examples: 200
Number of testing examples: 100


In [12]:
model_ckpt = "MCG-NJU/videomae-base" # pre-trained model from which to fine-tune
batch_size = 4 # batch size for training and evaluation

In [13]:
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification
image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt)
model = VideoMAEForVideoClassification.from_pretrained(
    model_ckpt,
    label2id=label2id,
    id2label=id2label
)

2024-03-17 19:35:11.122289: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 19:35:11.122396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 19:35:11.298175: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/377M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import pytorchvideo.data

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


In [15]:
!pip install pytorchvideo

In [16]:
import os

mean = image_processor.image_mean
std = image_processor.image_std
if "shortest_edge" in image_processor.size:
    height = width = image_processor.size["shortest_edge"]
else:
    height = image_processor.size["height"]
    width = image_processor.size["width"]
resize_to = (height, width)

num_frames_to_sample = model.config.num_frames
sample_rate = 4
fps = 30
clip_duration = num_frames_to_sample * sample_rate / fps


# Training dataset transformations.
train_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    RandomShortSideScale(min_size=256, max_size=320),
                    RandomCrop(resize_to),
                    RandomHorizontalFlip(p=0.5),
                ]
            ),
        ),
    ]
)

# Training dataset.
train_dataset = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "train"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("random", clip_duration),
    decode_audio=False,
    transform=train_transform,
)

# Validation and evaluation datasets' transformations.
val_transform = Compose(
    [
        ApplyTransformToKey(
            key="video",
            transform=Compose(
                [
                    UniformTemporalSubsample(num_frames_to_sample),
                    Lambda(lambda x: x / 255.0),
                    Normalize(mean, std),
                    Resize(resize_to),
                ]
            ),
        ),
    ]
)

# Validation and evaluation datasets.
val_dataset = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "val"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)

test_dataset = pytorchvideo.data.Ucf101(
    data_path=os.path.join(dataset_root_path, "test"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)

In [17]:
train_dataset.num_videos, val_dataset.num_videos, test_dataset.num_videos

(700, 200, 100)

In [18]:
from transformers import TrainingArguments, Trainer

model_name = model_ckpt.split("/")[-1]
new_model_name = f"{model_name}-finetuned-ucfcrime-full2"
num_epochs = 4

args = TrainingArguments(
    new_model_name,
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    max_steps=(train_dataset.num_videos // batch_size) * num_epochs,
)

In [19]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 870.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [20]:
import evaluate

metric = evaluate.load("accuracy")

In [21]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions."""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [22]:
import torch
import numpy as np

def collate_fn(examples):
    """The collation function to be used by `Trainer` to prepare data batches."""
    # permute to (num_frames, num_channels, height, width)
    pixel_values = torch.stack(
        [example["video"].permute(1, 0, 2, 3) for example in examples]
    )
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
train_results = trainer.train()

wandb: Currently logged in as: dumbal. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240317_193606-86yrafir
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run giddy-rain-28
wandb: ⭐️ View project at https://wandb.ai/dumbal/huggingface
wandb: 🚀 View run at https://wandb.ai/dumbal/huggingface/runs/86yrafir
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
0,2.583600,2.494397,0.207992
1,2.321200,2.585451,0.177259
2,2.233300,2.627028,0.104554
3,1.985000,2.405848,0.210894
4,2.194000,2.365350,0.223545
5,1.979600,2.260934,0.223545
6,1.878600,2.272467,0.234112
7,1.710000,2.222787,0.222578


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/

In [25]:
trainer.evaluate(test_dataset)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 2.50138521194458,
 'eval_accuracy': 0.225,
 'eval_runtime': 955.2551,
 'eval_samples_per_second': 5.192,
 'eval_steps_per_second': 0.649,
 'epoch': 7.12}

In [26]:
trainer.save_model()
test_results = trainer.evaluate(test_dataset)
trainer.log_metrics("test", test_results)
trainer.save_metrics("test", test_results)
trainer.save_state()
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1710704166.500f3adbbe43.26.0:   0%|          | 0.00/23.1k [00:00<?, ?B/s]

events.out.tfevents.1710729400.500f3adbbe43.26.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** test metrics *****
  epoch                   =       7.12
  eval_accuracy           =      0.225
  eval_loss               =     2.5014
  eval_runtime            = 0:15:45.27
  eval_samples_per_second =      5.247
  eval_steps_per_second   =      0.656


events.out.tfevents.1710729400.500f3adbbe43.26.1:   0%|          | 0.00/734 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/archit11/videomae-base-finetuned-ucfcrime-full2/commit/00306228351b830e6403a5a71485f51d96b43525', commit_message='End of training', commit_description='', oid='00306228351b830e6403a5a71485f51d96b43525', pr_url=None, pr_revision=None, pr_num=None)